In [ ]:
#pip install selenium

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url ='https://www.latam.com/es_pe/apps/personas/booking?fecha1_dia=14&fecha1_anomes=2020-10&fecha2_dia=30&fecha2_anomes=2020-11&from_city2=SCL&to_city2=LIM&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Lima&from_city1=LIM&vuelos_destino=Santiago%20de%20Chile&to_city1=SCL&flex=1&vuelos_fecha_salida_ddmmaaaa=15/10/2020&vuelos_fecha_regreso_ddmmaaaa=30/11/2020&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo=&stopover_outbound_days=0&stopover_inbound_days=0&application=#/'

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}

In [ ]:
r = requests.get(url, headers=headers)

In [ ]:
r.status_code

In [ ]:
print(r.request.headers)

In [ ]:
s = BeautifulSoup(r.text,'lxml')

In [ ]:
print(s.prettify())

In [ ]:
from selenium import webdriver

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='./chromedriver' , options=options)

In [ ]:
driver.get(url)

In [ ]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
vuelos

In [ ]:
vuelo = vuelos[0]

In [ ]:
vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [ ]:
vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [ ]:
vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

In [ ]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]').find_element_by_xpath('.//button[@class="sc-bdVaJa fuucJY"]')

In [ ]:
boton_escalas

In [ ]:
boton_escalas.click()

In [ ]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')

In [ ]:
escalas = len(segmentos)
escalas

In [ ]:
segmento = segmentos[0]

In [ ]:
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]').text

In [ ]:
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')

In [ ]:
segmento.find_elements_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]')[1].text

In [ ]:
driver.find_element_by_xpath('//button[@class="close"]').click()

In [ ]:
vuelo.click()

In [ ]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class,"fare-")]')

In [ ]:
tarifas

In [ ]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
    dict_tarifa = {nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

## Función para obtener precios de un vuelo buscado

In [ ]:
def obtener_precios(vuelo):
    '''
    Función que retorna una lista de diccionarios con las distintas tarifas
    '''
    tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class,"fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
        dict_tarifa = {nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
        #print(dict_tarifa)
    return precios 

## Función para obtener escalas de cada vuelo

In [ ]:
def obtener_datos_escalas(vuelo):
    '''
    Función que retorna una lista de diccionarios con la información de las escalas de cada vuelo
    '''
    segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
    info_escalas = []
    for segmento in segmentos:
        #Origen
        origen = segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]').text
        #Hora de salida
        dep_time = segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').get_attribute('datetime')
        #Destino
        destino = segmento.find_elements_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]')[1].text
        #Hora de llegada
        arr_time = ''#segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time')[1].get_attribute('datetime')
        #Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath('.//span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')
        #Número de vuelo
        numero_vuelo = segmento.find_element_by_xpath('.//div[@class="airline-flight-details"]/b').text
        #Modelo de avión
        modelo_avion = segmento.find_element_by_xpath('.//div[@class="airline-flight-details"]/span[@class="sc-gzOgki uTyOl"]').text
        if segmento != segmentos[-1]:
            #Duración de la escala
            duracion_escala = segmento.find_element_by_xpath('.//span[@class="sc-esjQYD dMquDU"]/time').get_attribute('datetime')
        else:
            duracion_escala = ''
        
        data_dict = {
            'origen':origen,
            'dep_time':dep_time,
            'destino':destino,
            'arr_time':arr_time,
            'duracion_vuelo':duracion_vuelo,
            'numero_vuelo':numero_vuelo,
            'modelo_avion':modelo_avion,
            'duracion_escala':duracion_escala
            
        }
        
        info_escalas.append(data_dict)
        return info_escalas

## Función para obtener tiempos de cada vuelo

In [ ]:
def obtener_tiempos(vuelo):
    '''
    Función que retorna un diccionario con los horarios de salid y llegada de cada vuelo, incluyendo la duración
    '''
    #Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    #Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    #Duracion 
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    tiempos = {'hora_salida':salida,'hora_llegada':llegada,'duracion':duracion}
    
    return tiempos    

## Consolida funciones para obtener resultado

In [ ]:
def obtener_info(driver):
   
    #driver.find_element_by_xpath('.//div[@class="onesignal-slidedown-dialog"]//button[@class="align-right secondary slidedown-button"]').click()
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    info = []
    
    for vuelo in vuelos:
        #Obtenemos tiempos de vuelo
        tiempos = obtener_tiempos(vuelo)
        #Obtenemos datos de las escalas
        vuelo.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]').find_element_by_xpath('.//button[@class="sc-bdVaJa fuucJY"]').click()
        escalas = obtener_datos_escalas(vuelo)
        #Cerramos el modal de las escalas
        driver.find_element_by_xpath('.//button[@class="close"]').click()
        #Clickeamos el vuelo para ver los precios 
        vuelo.click()
        #Obtenemos precios de los vuelos
        precios = obtener_precios(vuelo)
        vuelo.click()
        info.append({'precios':precios,'tiempos':tiempos,'escalas':escalas})
        
    return info

## Ejecuta url -> latam

In [ ]:
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='./chromedriver' , options=options)
driver.get(url)
time.sleep(5)
driver.find_element_by_xpath('.//div[@class="onesignal-slidedown-dialog"]//button[@class="align-right secondary slidedown-button"]').click()
#Introducir una demora
#time.sleep(10)
delay = 10
try:
    #introducir demora inteligente
    vuelo = WebDriverWait(driver , delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La pagina terminó de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La pagina tardó demasiado en cargar')
    info_vuelos = []
#obtener_info(driver)

## Obtiene resultado

In [ ]:
info_vuelos

In [ ]:
obtener_info(driver)

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='./chromedriver' , options=options)
driver.get(url)
time.sleep(5)
driver.find_element_by_xpath('.//div[@class="onesignal-slidedown-dialog"]//button[@class="align-right secondary slidedown-button"]').click()

In [ ]:
driver.close()